<a href="https://colab.research.google.com/github/pjlean7/Graficos_EIV/blob/main/Analise_de_dados_rotina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importando bibliotecas necessárias**

In [ ]:
import unicodedata

In [ ]:
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

## **Importando documento e fazendo edições necessárias na tabela**

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1wWc-Q7MPe_GC5RGyjlZoLYMZhy7bG_2stNxbsBtggNY/export?format=xlsx'

In [ ]:
dados_cabecalho = pd.read_excel(url, nrows=0)

In [ ]:
dados_cabecalho

In [ ]:
## Pulei 192 linhas pois estava buscando dados mais atualizados onde eu ja tinha colocado as informações corretamente ( os anteriores estavam para teste)
dados = pd.read_excel(url, skiprows = 192)
dados.head()

In [ ]:
## Colocar os nomes corretos nas colunas
dados.columns = dados_cabecalho.columns
dados.head()

In [ ]:
## Colcoando o as informações de data/hora da maneira correta para editar
dados['data/hora'] = pd.to_datetime(dados['data/hora'])
dados

# **Limpando dados nulos**

In [ ]:
dados.isnull().sum()

In [ ]:
dados.shape

In [ ]:
dados = dados.dropna(axis=0)
dados

In [ ]:
dados.shape

In [ ]:
dados.tail()

## **Separando os últimos 7 dias**

In [ ]:
## Eesetando a planilha dados_filtrado_7dias
dados_filtrado_7dias = pd.DataFrame()
dados_filtrado_7dias

In [ ]:
#Definir o hoje
hoje = dt.date(2025,11,27)
hoje

In [ ]:
#1. Definimos o limite (hoje menos 7 dias)
limite = hoje - dt.timedelta(days=7)

#2. Formantando o limite e o dia de hoje
limite = pd.to_datetime(limite)
hoje = pd.to_datetime(hoje)

#3. Filtramos a tabela: apenas linhas onde a data é maior ou igual ao limite (utilizamos o codigo **copy** no final para que seja criadao um novo arquivo desta tabela e não tenha nenhum problema futuro)
dados_filtrado_7dias = dados[(dados['data/hora'] >= limite) & (dados['data/hora'] <= hoje)].copy()
dados_filtrado_7dias

In [ ]:
## Reduzindo as datas para apenas dia e mês
dados_filtrado_7dias['data/hora'] = pd.to_datetime(dados_filtrado_7dias['data/hora']).dt.strftime('%d/%m')
dados_filtrado_7dias

# **Criando data frame meta (água)**

In [ ]:
agua = {
    'descricao': ['Água','Água','Água','Água','Água','Água','Água'],
    'quantidade':[10,10,10,10,10,10,10],
    'unidade': ['copo','copo','copo','copo','copo','copo','copo'],
    'ml': [2000,2000,2000,2000,2000,2000,2000],
    'dia_semana': ['domingo', 'segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado' ]
}
meta_agua = pd.DataFrame(agua)
meta_agua

# **Água ultimos 7 dias**

In [ ]:
## Selecionando a categoria desejada (tipo)
dados_liquido = dados_filtrado_7dias.query('tipo == "Bebida"')
dados_liquido

In [ ]:
dados_liquido['data/hora'].unique()

In [ ]:
## Moldando o grupo da categoria selecionada com as colunas que desejo
dados_liquido = dados_liquido.groupby(['data/hora', 'descricao', 'unidade','dia_semana'])['quantidade'].sum().reset_index()

# Converter a coluna 'data/hora' para datetime objects temporariamente para permitir a ordenação
dados_liquido['data_para_ordenar'] = pd.to_datetime(dados_liquido['data/hora'], format='%d/%m', errors='coerce')

# Ordenar o DataFrame pelo 'data_para_ordenar'
dados_liquido.sort_values(by='data_para_ordenar', inplace=True)

# Remover a coluna temporária e resetar o índice
dados_liquido.drop(columns='data_para_ordenar', inplace=True)
dados_liquido.reset_index(drop=True, inplace=True)
dados_liquido

In [ ]:
## Cortando o refrigerante da tabela
linhas_a_remover = dados_liquido.query('descricao == 	"Refrigerante"').index

dados_liquido.drop(linhas_a_remover, inplace=True)
dados_liquido

In [ ]:
## Colocando toda as quantidades em ml
copo = 250
dados_liquido['ml'] = dados_liquido['quantidade'] * copo
dados_liquido

#**Gráficos água**

In [ ]:
plt.figure(figsize=(5, 5))
sns.set_theme(style="darkgrid")

ax = sns.barplot(
    data = dados_liquido,
    x = 'data/hora',
    y = 'ml',
    hue = 'descricao',
    palette = ['#4C72B0']
    )
ax.bar_label(ax.containers[0], label_type='center', color='#403B3B', fontweight='bold', fmt='%.0f', fontsize= 10)
plt.title('Consumo de Água semanal')
plt.xlabel('Data (Dia/Mês)')
plt.ylabel('Quantidade (ml)')
plt.legend()

plt.show()

In [ ]:
#Gerando gráfico
sns.set_theme(style="white")
plt.figure(figsize=(10, 6))

#Gráfico meta
ax = sns.barplot(
    data=meta_agua,
    x='dia_semana',
    y='ml',
    color='#8EB2FA',
    alpha = 0.3,
    label='Meta Estipulada',
    width=0.5
)
#Gráfico real
sns.barplot(
    data=dados_liquido,
    x='dia_semana',
    y='ml',
    color='#4C72B0',
    label='Consumo Real',
    width=0.5
)
#Configurações estéticas do grafico
ax.bar_label(ax.containers[1], label_type='center', color='#403B3B', fontweight='bold', fmt='%.0f', fontsize= 10)
plt.title('Consumo de Água: Real vs Meta')
plt.xlabel('Dia da semana')
plt.ylabel('Quantidade (ml)')
plt.legend()

plt.show()

In [ ]:
media_agua_semanal = dados_liquido['ml'].sum()/7
media_agua_semanal

# **Criando data frame meta (suplementos)**

In [ ]:
suplemento = {
    'descricao': ['Whey','Creatina', 'Whey','Creatina','Whey','Creatina','Whey','Creatina','Whey','Creatina','Whey','Creatina','Whey','Creatina'],
    'quantidade':[1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'unidade': ['unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade','unidade',],
    'dia_semana': ['domingo','domingo', 'segunda-feira','segunda-feira', 'terça-feira','terça-feira', 'quarta-feira','quarta-feira', 'quinta-feira','quinta-feira', 'sexta-feira','sexta-feira', 'sábado', 'sábado']
}
meta_suplemento = pd.DataFrame(suplemento)
meta_suplemento

# **Suplementação 7 dias**

In [ ]:
dados_suplemento = dados_filtrado_7dias.query('tipo == "Suplemento"')
dados_suplemento

In [ ]:
dados_suplemento = dados_suplemento.groupby([ 'data/hora', 'descricao', 'unidade','dia_semana'])['quantidade'].sum().reset_index()
dados_suplemento

In [ ]:
sns.set_theme(style="darkgrid")
plt.figure(figsize=(14, 7))

# 3. Criação do Gráfico de Barras Agrupadas
ax = sns.barplot(
    data=dados_suplemento,
    x='data/hora',
    y='quantidade',
    hue='descricao',
    palette=[ '#66342B','#4C72B0'] # Cores customizadas
)

# 4. Adicionar os nomes dentro das barrinhas
for p in ax.patches:
    if p.get_height() > 0:
        # Pega o label (Whey ou Creatina) baseado na cor/legenda
        # Para simplificar no matplotlib puro, identificamos a barra
        label = "Creatina" if p.get_x() % 1 < 0.5 else "Whey"

        ax.annotate(
            label,
            (p.get_x() + p.get_width() / 2., p.get_height() / 2),
            ha='center', va='center',
            fontsize=9, color='white', weight='bold',
            rotation=90
        )
handles, labels = ax.get_legend_handles_labels()
# 5. Ajustes de layout e títulos
plt.title('Consumo Diário: Bebida Whey vs Creatina', fontsize=16, pad=20)
plt.xlabel('Data (Dia/Mês)', fontsize=12)
plt.ylabel('Quantidade Consumida', fontsize=12)
plt.xticks(rotation=0)
plt.legend(handles, ["Whey", "Creatina"], title='Suplemento', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# 6. Exibir ou Salvar
plt.show()

In [ ]:
sns.set_theme(style="darkgrid")
plt.figure(figsize=(14, 7))

# --- 1. DESENHAR A META (ATRÁS) ---
# Usamos uma largura (width) maior e opacidade (alpha) baixa
ax = sns.barplot(
    data=meta_suplemento,
    x='dia_semana',
    y='quantidade',
    hue='descricao',
    palette=['#66342B','#4C72B0'],
    alpha=0.3,      # Bem clarinho para parecer fundo
    edgecolor='black', # Linha fina para definir o limite da meta
    linewidth=0.5
)

# --- 2. DESENHAR O REAL (NA FRENTE) ---
# Usamos a mesma paleta, mas com cores sólidas e barras mais finas
sns.barplot(
    data=dados_suplemento,
    x='dia_semana',
    y='quantidade',
    hue='descricao',
    palette=[ '#66342B','#4C72B0'],
    alpha=1.0,
    ax=ax # Garante que desenha no mesmo gráfico
)

# --- 3. LÓGICA DE NOMES PARA TODAS AS BARRAS ---
# Temos 4 containers agora: [0]=Whey Meta, [1]=Creatina Meta, [2]=Whey Real, [3]=Creatina Real
nomes = ["Whey", "Creatina", "Whey", "Creatina"]

for i, container in enumerate(ax.containers):
    label_nome = nomes[i]

    ax.bar_label(
        container,
        # O label aparece se o valor da barra for maior que zero
        labels=[label_nome if v > 0 else "" for v in container.datavalues],
        label_type='center',
        rotation=90,
        color='white' if i >= 2 else 'gray', # Branco no real, cinza na meta
        weight='bold',
        fontsize=9
    )
# --- 4. AJUSTES DE LAYOUT ---
plt.title('Consumo vs Meta: Whey e Creatina', fontsize=16, pad=20)
plt.xlabel('Dia da semana', fontsize=12)
plt.ylabel('Quantidade Consumida', fontsize=12)

# Ajuste da Legenda para não duplicar itens
handles, labels = ax.get_legend_handles_labels()
# Pegamos apenas os primeiros 2 itens da legenda (Whey e Creatina)
plt.legend(handles[2:],["Whey", "Creatina"], title='Suplemento', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

# **Criando data frame meta (cafe da manha, almoço, lanche e jantar)**

##**Cafe da manha (meta)**

In [ ]:
cafe_da_manha = {
    'tipo':['Carboidrato', 'Leite_derivados','Leite_derivados','Frutas'],
    'descricao': ['Pão', 'Queijo', 'Iogurte', 'Frutas congeladas' ],
    'quantidade':[1, 50, 170, 1],
    'unidade': ['unidade', 'gramas', 'gramas', 'porcao']
}
meta_cafe_da_manha = pd.DataFrame(cafe_da_manha)
meta_cafe_da_manha

##**Almoço (meta)**

In [ ]:
almoco = {
    'tipo':['Carboidrato', 'Leguminosas', 'Carne'],
    'quantidade': [170, 200, 120],
    'unidade': ['gramas', 'gramas', 'gramas']
}
meta_almoco = pd.DataFrame(almoco)
meta_almoco

##**Lanche da tarde (meta)**

In [ ]:
lanche = {
    'descricao':['Pão', 'Requeijão', 'Ovo'],
    'quantidade': [1, 70, 2],
    'unidade': ['unidade', 'gramas', 'unidade']
}
meta_lanche = pd.DataFrame(lanche)
meta_lanche

##**Jantar (meta)**

In [ ]:
jantar = {
    'tipo':['Carboidrato', 'Leguminosas', 'Carne'],
    'quantidade': [100, 100, 100],
    'unidade': ['gramas', 'gramas', 'gramas']
}
meta_jantar = pd.DataFrame(jantar)
meta_jantar

# **Café da manhã 7 dias**

In [ ]:
dados_cafe_da_manha = dados_filtrado_7dias.query('atividade == "refeicao_cafe_manha"')
dados_cafe_da_manha

In [ ]:
lista_dia_semana = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira']
lista_titulo_semana_dia = ['segunda_feira', 'terca_feira', 'quarta_feira', 'quinta_feira', 'sexta_feira']

In [ ]:
for i in lista_dia_semana:
    dados_filtrado = dados_cafe_da_manha.query("dia_semana == @i")
    # 2. Limpa o nome para virar uma variável válida:
    # Remove acentos
    nfkd_form = unicodedata.normalize('NFKD', i)
    nome_limpo = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Substitui espaços e hifens por underline e deixa tudo minúsculo
    nome_limpo = nome_limpo.lower().replace(" ", "_").replace("-", "_")

    nome_variavel = f"dados_cafe_da_manha_{nome_limpo}"

    # 3. Cria a variável global com esse nome
    globals()[nome_variavel] = dados_filtrado
    print(f"Variável criada: {nome_variavel}")



# **Gráficos café da manhã**

In [ ]:
for i in lista_titulo_semana_dia:
    # Monta o nome da variável exatamente como ela existe no ambiente
    nome_variavel = f"dados_cafe_da_manha_{i}"

    # Verifica se a variável existe para evitar erros
    if nome_variavel in globals():
        # Acessa o DataFrame dinamicamente
        df_do_dia = globals()[nome_variavel]

        sns.set_theme(style="white")
        plt.figure(figsize=(10, 6))

        # Gráfico meta (fixo)
        ax = sns.barplot(
            data=meta_cafe_da_manha,
            x='descricao',
            y='quantidade',
            color='#8EB2FA',
            alpha=0.3,
            label='Meta Estipulada',
            width=0.6
        )

        # Gráfico real (dinâmico baseado no dia)
        sns.barplot(
            data=df_do_dia,
            x='descricao',
            y='quantidade',
            color='#4C72B0',
            label='Consumo Real',
            width=0.55
        )

        # Configurações estéticas
        # O título agora usa 'i' para identificar o dia
        plt.title(f'Café da manhã {i.replace("_", " ").title()}: Real vs Meta', fontsize=14)
        plt.xlabel('Tipo de Alimentos')
        plt.ylabel('Quantidade')

        # Ajuste para garantir que os labels apareçam no lugar certo
        if len(ax.containers) > 1:
            ax.bar_label(ax.containers[1], label_type='center', color='black', fontweight='bold', fmt='%.0f')

        plt.legend()
        plt.show()
    else:
        print(f"Erro: A variável {nome_variavel} não foi encontrada no ambiente.")

# **Almoço 7 dias**

In [ ]:
dados_almoco = dados_filtrado_7dias.query('atividade == "refeicao_almoco"')
dados_almoco

In [ ]:

for i in lista_dia_semana:
    dados_filtrado = dados_almoco.query("dia_semana == @i")
    # 2. Limpa o nome para virar uma variável válida:
    # Remove acentos
    nfkd_form = unicodedata.normalize('NFKD', i)
    nome_limpo = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Substitui espaços e hifens por underline e deixa tudo minúsculo
    nome_limpo = nome_limpo.lower().replace(" ", "_").replace("-", "_")

    nome_variavel = f"dados_almoco_{nome_limpo}"

    # 3. Cria a variável global com esse nome
    globals()[nome_variavel] = dados_filtrado
    print(f"Variável criada: {nome_variavel}")

# **Gráficos Almoço**

In [ ]:
for i in lista_titulo_semana_dia:
    # Monta o nome da variável exatamente como ela existe no ambiente
    nome_variavel = f"dados_almoco_{i}"

    # Verifica se a variável existe para evitar erros
    if nome_variavel in globals():
        # Acessa o DataFrame dinamicamente
        df_do_dia = globals()[nome_variavel]

        sns.set_theme(style="white")
        plt.figure(figsize=(10, 6))

        # Gráfico meta (fixo)
        ax = sns.barplot(
            data=meta_almoco,
            x='tipo',
            y='quantidade',
            color='#8EB2FA',
            alpha=0.3,
            label='Meta Estipulada',
            width=0.6
        )

        # Gráfico real (dinâmico baseado no dia)
        sns.barplot(
            data=df_do_dia,
            x='tipo',
            y='quantidade',
            color='#4C72B0',
            label='Consumo Real',
            width=0.55
        )

        # Configurações estéticas
        # O título agora usa 'i' para identificar o dia
        plt.title(f'Almoço {i.replace("_"," ").title()}: Real vs Meta', fontsize=14)
        plt.xlabel('Tipo de Alimentos')
        plt.ylabel('Quantidade')

        # Ajuste para garantir que os labels apareçam no lugar certo
        if len(ax.containers) > 1:
            ax.bar_label(ax.containers[1], label_type='center', color='black', fontweight='bold', fmt='%.0f')

        plt.legend()
        plt.show()
    else:
        print(f"Erro: A variável {nome_variavel} não foi encontrada no ambiente.")

# **Lanche 7 dias**

In [ ]:
dados_lanche = dados_filtrado_7dias.query('atividade == "refeicao_tarde"')
dados_lanche

In [ ]:
for i in lista_dia_semana:
    dados_filtrado = dados_lanche.query("dia_semana == @i")

    # 2. Limpa o nome para virar uma variável válida:
    # Remove acentos
    nfkd_form = unicodedata.normalize('NFKD', i)
    nome_limpo = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Substitui espaços e hifens por underline e deixa tudo minúsculo
    nome_limpo = nome_limpo.lower().replace(" ", "_").replace("-", "_")

    nome_variavel = f"dados_lanche_{nome_limpo}"

    # 3. Cria a variável global com esse nome
    globals()[nome_variavel] = dados_filtrado
    print(f"Variável criada: {nome_variavel}")


# **Gráficos lanche da tarde**

In [ ]:
for i in lista_titulo_semana_dia:
  nome_variavel = f'dados_lanche_{i}'

  if nome_variavel in globals():
    df_do_dia = globals()[nome_variavel]

    sns.set_theme(style="white")
    plt.figure(figsize=(10, 6))

  ax= sns.barplot(
      data=meta_lanche,
      x='descricao',
      y='quantidade',
      color='#8EB2FA',
      alpha=0.3,
      label='Meta Estipulada',
      width=0.6
  )

  sns.barplot(
      data=df_do_dia,
      x='descricao',
      y='quantidade',
      color='#4C72B0',
      label='Consumo Real',
      width=0.55
  )

  plt.title(f'Lanche da tarde {i.replace("_", " ").title()}: Real vs Meta', fontsize=14)
  plt.xlabel('Tipo de Alimentos')
  plt.ylabel('Quantidade')

  if len(ax.containers) > 1:
            ax.bar_label(ax.containers[1], label_type='center', color='black', fontweight='bold', fmt='%.0f')

  plt.legend()
  plt.show()
else:
        print(f"Erro: A variável {nome_variavel} não foi encontrada no ambiente.")

# **Jantar 7 dias**

In [ ]:
dados_jantar = dados_filtrado_7dias.query('atividade == "refeicao_jantar"')
dados_jantar

In [ ]:
for i in lista_dia_semana:
  dados_filtrados = dados_jantar.query("dia_semana == @i")

  # 2. Limpa o nome para virar uma variável válida:
  # Remove acentos
  nfkd_form = unicodedata.normalize('NFKD', i)
  nome_limpo = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

  # Substitui espaços e hifens por underline e deixa tudo minúsculo
  nome_limpo = nome_limpo.lower().replace(" ", "_").replace("-", "_")

  nome_variavel = f"dados_jantar_{nome_limpo}"

  globals()[nome_variavel] = dados_filtrados
  print(f'Variável criada {nome_variavel}')


In [ ]:

for i in lista_titulo_semana_dia:
  nome_variavel = f'dados_jantar_{i}'

  if nome_variavel in globals():
    df_do_dia = globals()[nome_variavel]

    sns.set_theme(style="white")
    plt.figure(figsize=(10, 6))

    ax = sns.barplot(
        data=meta_jantar,
        x='tipo',
        y='quantidade',
        color='#8EB2FA',
        alpha=0.3,
        label='Meta Estipulada',
        width=0.6
    )
    sns.barplot(
        data=df_do_dia,
        x='tipo',
        y='quantidade',
        color='#4C72B0',
        label='Consumo Real',
        width=0.55
    )
    plt.title(f'Jantar {i.replace("_"," ").title()}: Real Vs Meta', fontsize=14)
    plt.xlabel('Tipo de Alimentos')
    plt.ylabel('Quantidade')
    if len(ax.containers) > 1:
      ax.bar_label(ax.containers[1], label_type='center', color='black', fontweight='bold', fmt='%.0f')

    plt.legend()
    plt.show()
  else:
    print(f"Erro: A variável {nome_variavel} não foi encontrada no ambiente.")